In [2]:
from fragsys_ml import *

2024-10-17 16:08:41.448478: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [66]:
input_id = "CMTR1_copy"
results_dir = f'./output/{input_id}/results'
bss_data_out = os.path.join(results_dir, f'{input_id}_bss_table.pkl')
bss_data_out_RSA = os.path.join(results_dir, f'{input_id}_bss_RSA_table.pkl')
rsa_profs_out = os.path.join(results_dir, f'{input_id}_bss_RSA_profiles.pkl')
bss_data = pd.read_pickle(bss_data_out)
rsa_profs = load_pickle(rsa_profs_out)

In [7]:
len(bss_data)

4

In [46]:
# rsa_profs = {f'{input_id}_{k}': v for k, v in rsa_profs.items()}

In [18]:
bss_data

,ID,RSA,DS,MES,Size,Cluster,FS
0,0,37.3,1.8,1.10,5,-1,-1
1,1,28.5,2.4,1.76,6,-1,-1
2,2,30.3,2.0,1.39,11,-1,-1
3,3,7.6,6.4,0.64,12,-1,-1


In [52]:
rsa_profs_ks = list(rsa_profs.keys())

In [53]:
bss_labs = bss_data.ID.tolist()

In [54]:
set(rsa_profs)

{0, 1, 2, 3}

In [56]:
len(set(rsa_profs_ks) - set(bss_labs))

0

In [57]:
rsa_profs_filt = {k:v for k, v in rsa_profs.items() if len(v) > 0}

In [58]:
len(rsa_profs_filt) # (lost 18 profiles of length 0)

4

In [59]:
max_len = max([len(v) for v in rsa_profs_filt.values()])
bs_vectors = []
bs_vectors_dict = {}
for bs_id, rsa_sig in rsa_profs_filt.items():
    rsa_sig_len = len(rsa_sig)
    rsa_range_prop = [0 for i in range(10)] # now let us change to 10
    for rsa in rsa_sig:
        prop_i = int(rsa/10) # 10 RSA BINS: b1 = [0,10), b2 = [10, 20), ... b10 = [90, MAX)
        if prop_i > 9: # if greater than 100, put in 10th bin
            prop_i = 9
        #print(prop_i)
        rsa_range_prop[prop_i] += 1
    rsa_range_prop = [round(i/rsa_sig_len, 3) for i in rsa_range_prop]
    rsa_range_prop.insert(0, rsa_sig_len/max_len) # ADDING BINDING SITE SIZE RELATIVE TO MAX SITE SIZE (IN THIS CASE 40)
    bs_vectors.append(rsa_range_prop)
    bs_vectors_dict[bs_id] = rsa_range_prop
    
vector_df = pd.DataFrame(bs_vectors, index = list(rsa_profs_filt.keys())) # obtaining RSA vectors, which are the 11-element features used for the machine learning

In [60]:
vector_df

,0,1,2,3,4,5,6,7,8,9,10
0,0.416667,0.200,0.200,0.000,0.000,0.200,0.200,0.200,0.000,0.0,0.0
1,0.500000,0.333,0.167,0.000,0.167,0.167,0.000,0.000,0.167,0.0,0.0
2,0.916667,0.273,0.182,0.182,0.091,0.000,0.091,0.091,0.091,0.0,0.0
3,1.000000,0.667,0.250,0.083,0.000,0.000,0.000,0.000,0.000,0.0,0.0


In [61]:
model_path = "/homes/2394007/JSU_LIGYSIS/ANN_results/diff_seeds/85406/models/85406_4_model_epoch_75_train_acc_0.88.h5"
final_model = keras.models.load_model(model_path)

final_preds = final_model.predict(x = vector_df, batch_size = 27, verbose = 0)

rounded_predictions = np.argmax(final_preds, axis = -1)

site_names = vector_df.index.tolist()

clab_dict = {}
for i in range(len(vector_df)):
    clab_dict[site_names[i]] = rounded_predictions[i]+1
    
for i in [0,1,2,3]:
    print(i, list(rounded_predictions).count(i))
    
func_scores_dict = {}
for i in range(len(final_preds)):
    func_score = ((0.52*final_preds[i][0])+(0.18*final_preds[i][1])+(0.05*final_preds[i][2])+(0.04*final_preds[i][3]))
    func_scores_dict[site_names[i]] = round(func_score,2)

0 3
1 1
2 0
3 0


In [62]:
clab_dict

{0: 2, 1: 1, 2: 1, 3: 1}

In [41]:
bss_data

,ID,RSA,DS,MES,Size,Cluster,FS
0,0,37.3,1.8,1.10,5,-1,-1
1,1,28.5,2.4,1.76,6,-1,-1
2,2,30.3,2.0,1.39,11,-1,-1
3,3,7.6,6.4,0.64,12,-1,-1


In [63]:
bss_data["Cluster"] = bss_data.ID.map(clab_dict)
bss_data["FS"] = bss_data.ID.map(func_scores_dict)

In [9]:
bss_data

,ID,RSA,DS,MES,Size,Cluster,FS
0,0,37.3,1.8,1.10,5,NaN,0.14
1,1,28.5,2.4,1.76,6,NaN,0.37
2,2,30.3,2.0,1.39,11,NaN,0.38
3,3,7.6,6.4,0.64,12,NaN,0.52


In [68]:
bss_data.to_pickle(bss_data_out_RSA)

## Tidying it up for script

In [11]:
import os
import pickle
import numpy as np
import pandas as pd
from tensorflow import keras

def load_pickle(f_in):
    """
    loads pickle
    """
    with open(f_in, "rb") as f:
        data = pickle.load(f)
    return data

def get_RSA_vectors(rsa_profs_filt):
    max_len = max([len(v) for v in rsa_profs_filt.values()])
    bs_vectors = []
    bs_vectors_dict = {}
    for bs_id, rsa_sig in rsa_profs_filt.items():
        rsa_sig_len = len(rsa_sig)
        rsa_range_prop = [0 for i in range(10)] # now let us change to 10
        for rsa in rsa_sig:
            prop_i = int(rsa/10) # 10 RSA BINS: b1 = [0,10), b2 = [10, 20), ... b10 = [90, MAX)
            if prop_i > 9: # if greater than 100, put in 10th bin
                prop_i = 9
            rsa_range_prop[prop_i] += 1
        rsa_range_prop = [round(i/rsa_sig_len, 3) for i in rsa_range_prop]
        rsa_range_prop.insert(0, rsa_sig_len/max_len) # ADDING BINDING SITE SIZE RELATIVE TO MAX SITE SIZE (IN THIS CASE 40)
        bs_vectors.append(rsa_range_prop)
        bs_vectors_dict[bs_id] = rsa_range_prop

    vector_df = pd.DataFrame(bs_vectors, index = list(rsa_profs_filt.keys())) # obtaining RSA vectors, which are the 11-element features used for the machine learning

    return vector_df

input_id = "CMTR1_copy"

results_dir = f'./output/{input_id}/results'
bss_data_out = os.path.join(results_dir, f'{input_id}_bss_table.pkl')
bss_data_out_RSA = os.path.join(results_dir, f'{input_id}_bss_RSA_table.pkl')
rsa_profs_out = os.path.join(results_dir, f'{input_id}_bss_RSA_profiles.pkl')

model_path = "/homes/2394007/JSU_LIGYSIS/ANN_results/diff_seeds/85406/models/85406_4_model_epoch_75_train_acc_0.88.h5"

bss_data = pd.read_pickle(bss_data_out)
rsa_profs = load_pickle(rsa_profs_out)
rsa_profs_ks = list(rsa_profs.keys())
bss_labs = bss_data.ID.tolist()
rsa_profs_filt = {k:v for k, v in rsa_profs.items() if len(v) > 0}

vector_df = get_RSA_vectors(rsa_profs_filt)

final_model = keras.models.load_model(model_path)
final_preds = final_model.predict(x = vector_df, batch_size = 27, verbose = 0)
rounded_predictions = np.argmax(final_preds, axis = -1)
site_names = vector_df.index.tolist()

clab_dict = {}
for i in range(len(vector_df)):
    clab_dict[site_names[i]] = rounded_predictions[i]+1

func_scores_dict = {}
for i in range(len(final_preds)):
    func_score = ((0.52*final_preds[i][0])+(0.18*final_preds[i][1])+(0.05*final_preds[i][2])+(0.04*final_preds[i][3]))
    func_scores_dict[site_names[i]] = round(func_score,2)
    
bss_data["Cluster"] = bss_data.ID.map(clab_dict)
bss_data["FS"] = bss_data.ID.map(func_scores_dict)

bss_data.to_pickle(bss_data_out_RSA)

In [10]:
clab_dict

{}

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
from tensorflow import keras
import argparse

def load_pickle(f_in):
    """
    loads pickle
    """
    with open(f_in, "rb") as f:
        data = pickle.load(f)
    return data

def get_RSA_vectors(rsa_profs_filt):
    max_len = max([len(v) for v in rsa_profs_filt.values()])
    bs_vectors = []
    bs_vectors_dict = {}
    for bs_id, rsa_sig in rsa_profs_filt.items():
        rsa_sig_len = len(rsa_sig)
        rsa_range_prop = [0 for i in range(10)]  # now let us change to 10
        for rsa in rsa_sig:
            prop_i = int(rsa / 10)  # 10 RSA BINS: b1 = [0,10), b2 = [10, 20), ... b10 = [90, MAX)
            if prop_i > 9:  # if greater than 100, put in 10th bin
                prop_i = 9
            rsa_range_prop[prop_i] += 1
        rsa_range_prop = [round(i / rsa_sig_len, 3) for i in rsa_range_prop]
        rsa_range_prop.insert(0, rsa_sig_len / max_len)  # ADDING BINDING SITE SIZE RELATIVE TO MAX SITE SIZE (IN THIS CASE 40)
        bs_vectors.append(rsa_range_prop)
        bs_vectors_dict[bs_id] = rsa_range_prop

    vector_df = pd.DataFrame(bs_vectors, index=list(rsa_profs_filt.keys()))  # obtaining RSA vectors, which are the 11-element features used for the machine learning

    return vector_df

def main(input_id):
    results_dir = f'./output/{input_id}/results'
    bss_data_out = os.path.join(results_dir, f'{input_id}_bss_table.pkl')
    bss_data_out_RSA = os.path.join(results_dir, f'{input_id}_bss_RSA_table.pkl')
    rsa_profs_out = os.path.join(results_dir, f'{input_id}_bss_RSA_profiles.pkl')

    model_path = "/homes/2394007/JSU_LIGYSIS/ANN_results/diff_seeds/85406/models/85406_4_model_epoch_75_train_acc_0.88.h5"

    # Load data
    bss_data = pd.read_pickle(bss_data_out)
    rsa_profs = load_pickle(rsa_profs_out)
    rsa_profs_filt = {k: v for k, v in rsa_profs.items() if len(v) > 0}

    # Get RSA vectors
    vector_df = get_RSA_vectors(rsa_profs_filt)

    # Load the model
    final_model = keras.models.load_model(model_path)

    # Predict
    final_preds = final_model.predict(x=vector_df, batch_size=27, verbose=0)
    rounded_predictions = np.argmax(final_preds, axis=-1)
    site_names = vector_df.index.tolist()

    # Calculate functional scores
    func_scores_dict = {}
    for i in range(len(final_preds)):
        func_score = ((0.52 * final_preds[i][0]) + (0.18 * final_preds[i][1]) + (0.05 * final_preds[i][2]) + (0.04 * final_preds[i][3]))
        func_scores_dict[site_names[i]] = round(func_score, 2)

    # Add data to DataFrame
    bss_data["FS"] = bss_data.ID.map(func_scores_dict)

    # Save the modified DataFrame
    bss_data.to_pickle(bss_data_out_RSA)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Process input_id.')
    parser.add_argument('input_id', type=str, help='The input_id to process')
    args = parser.parse_args()

    main(args.input_id)
